In [1]:
import os
%pwd

's:\\study\\data scientist\\projects\\End-To-End MLOps\\MLOps\\research'

In [2]:
os.chdir("../")
%pwd

's:\\study\\data scientist\\projects\\End-To-End MLOps\\MLOps'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:

    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [30]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, scheme_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        self.scheme = read_yaml(str(scheme_filepath))

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        scheme = self.scheme.TARGET_COLUMN


        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=scheme.name
        )
        return model_trainer_config

In [31]:

import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [32]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=123)
        lr.fit(X_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [33]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:      
    logger.exception(e)
    raise e

[2025-06-07 00:22:23,115: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-07 00:22:23,116: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-07 00:22:23,118: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-07 00:22:23,120: INFO: common: Created directory at: artifacts]
[2025-06-07 00:22:23,120: INFO: common: Created directory at: artifacts/model_trainer]
